In [3]:
from bs4 import BeautifulSoup
import requests

In [53]:
# change this to the actual url and use requests to pull the page
index = "http://www.presidency.ucsb.edu/debates.php"
response = requests.get(index)

# beautiful soup object for html parsing
soup = BeautifulSoup(response.text.encode('utf-8'))

# find all links to debate transcripts
debate_links = soup.find_all('td', attrs={'class':'doctext'})
debate_dates = soup.find_all('td', attrs={'class':'docdate'})

In [63]:
#soup

In [64]:
#transcript_urls

In [56]:
# get urls for each transcript
transcript_urls = []
n_sans_link = 0
for i in range(len(debate_links)):
    try:
        transcript_urls.append(
            (debate_links[i].select('a')[0].get('href'), 
             debate_dates[i].text.split(', ')[1])
        )
    except IndexError:
        n_sans_link += 1
        
print 'got %i urls, %i listings with no link.' % (len(transcript_urls), n_sans_link)

got 127 urls, 20 listings with no link.


In [7]:
transcript_urls[-16][0]

'http://www.presidency.ucsb.edu/ws/index.php?pid=97332'

In [8]:
#response2.text[89135:]

In [9]:
response1 = requests.get(transcript_urls[0][0])
response2 = requests.get(transcript_urls[-16][0])

In [17]:
# html starts after a fat block of javascript and XML.
# read the page after that point

# new pages have the participants names in bold
start1 = response1.text.index('<span class="displaytext">')
end1 = response1.text.index('<hr noshade="noshade" size="1">')
soup1 = BeautifulSoup(response1.text.encode('utf-8'))

# old pages do not bold the participants names
start2 = response2.text.index('<span class="displaytext">')
end2 = response2.text.index('<hr noshade="noshade" size="1">')
soup2 = BeautifulSoup(response2.text[start2:end2])

In [22]:
import bs4
bs4.__version__

'4.4.1'

In [65]:
#soup1.find_all('p')

In [22]:
import re

In [58]:
def parse_speakers_and_quotes(page, year):
    """
    Extracts names of every person involved with the debate and 
    collects all of their quotes in a list.
    
    :param page: html page
    
    :returns dict: {speaker name: [quotes]}
    """
    #print year
    # fixes a BeautifulSoup problem that results in improper parsing
    page_soup = BeautifulSoup(page.text.encode('utf-8'))
    
    speaker_dict = dict()

    prev_speaker = ''
    for i in page_soup.find_all('p'):

        try: # search for name of person speaking
            curr_speaker = re.findall('[A-Z]+:', i.text)[0].strip(':')
            prev_speaker = curr_speaker
            quote = re.split(':', i.text)[1]

        except IndexError: # if name not in line
            quote = i.text

        if prev_speaker+'_'+year not in speaker_dict:
            # add speaker to speaker_dict with list of quotes as value
            speaker_dict[prev_speaker+'_'+year] = [quote]
        else:
            # append quote to speaker's list of quotes
            speaker_dict[prev_speaker+'_'+year].append(quote)
            
    return speaker_dict

In [59]:
from requests_futures.sessions import FuturesSession

npages = len(transcript_urls)

session = FuturesSession(max_workers=5)
futures = [session.get(url[0]) for url in transcript_urls[:npages]]

dates = [url[1] for url in transcript_urls[:npages]]
results = zip(futures, dates)

debate_transcripts = [parse_speakers_and_quotes(future.result(), date) for future, date in results]

In [61]:
import dill

In [62]:
dill.dump(debate_transcripts, open('debate_transcripts_list.dill', 'wb'))

In [66]:
#debate_transcripts